In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

# An example of a Low Liquidity Trade

In [8]:

import os
import requests
import s3fs
import pandas as pd
import re
import humps
import sqlalchemy as sa
from datetime import timedelta, datetime
import datetime as dt
import pendulum


import yfinance as yf

from prefect import Client, Task, Flow, task, Parameter
from prefect.schedules import IntervalSchedule
from prefect.environments.storage import Docker
from prefect.schedules import clocks, filters, Schedule
from prefect.environments.storage import Docker

# Homemade
from extractMOCData import CONFIG as cfg
from extractMOCData.moc_data import TsxMocData

from normalize.ticker_symbols import MapTickerSymbols
from addFeatures.daily import DailyData 

from mocIO.read_moc import get_moc_file_lst
#from extractMOCData import read_moc 

import etl_moc

## Add Scheduler

In [9]:
# Schedule when to run the script
schedule = Schedule(
    # fire every day
    clocks=[clocks.IntervalClock(
        start_date=pendulum.datetime(2020, 4, 22, 17, 15, tz="America/Toronto"),
        interval=timedelta(days=1)
        )],
    # but only on weekdays
    filters=[filters.is_weekday],

    # and not in January TODO: Add TSX Holidays
    not_filters=[filters.between_dates(1, 1, 1, 31)]
)

In [10]:
schedule.next(10)

[DateTime(2020, 4, 22, 17, 15, 0, tzinfo=Timezone('America/Toronto')),
 DateTime(2020, 4, 23, 17, 15, 0, tzinfo=Timezone('America/Toronto')),
 DateTime(2020, 4, 24, 17, 15, 0, tzinfo=Timezone('America/Toronto')),
 DateTime(2020, 4, 27, 17, 15, 0, tzinfo=Timezone('America/Toronto')),
 DateTime(2020, 4, 28, 17, 15, 0, tzinfo=Timezone('America/Toronto')),
 DateTime(2020, 4, 29, 17, 15, 0, tzinfo=Timezone('America/Toronto')),
 DateTime(2020, 4, 30, 17, 15, 0, tzinfo=Timezone('America/Toronto')),
 DateTime(2020, 5, 1, 17, 15, 0, tzinfo=Timezone('America/Toronto')),
 DateTime(2020, 5, 4, 17, 15, 0, tzinfo=Timezone('America/Toronto')),
 DateTime(2020, 5, 5, 17, 15, 0, tzinfo=Timezone('America/Toronto'))]

## Deploy to cloud with cloud compute resources


### 1. Dockerize the flow

There are two options to run the flow using docker, either compose your own base image .i.e. Dockerfile or use a build provided by prefect.  If the flow only has PyPI dependencies: then there is no need to compose a custom Docker file beyond what prefect provides under ther hood. [Dependencies](https://docs.prefect.io/orchestration/recipes/deployment.html#dependencies).
    
      a. Steps to compose a Dockerfile with a Flow 
            1. Create a new file called Dockerfile **with no extention**
            2. Add the following lines of code
    
      b. A

In [11]:
# Paths
working_dir_path = os.getcwd()
print(f"Working Dir: {working_dir_path}")

docker_flpth = os.path.join(working_dir_path, "Dockerfile")
print(f"Docker flpth: {docker_flpth}")

Working Dir: /home/ilivni/MOC
Docker flpth: /home/ilivni/MOC/Dockerfile


In [12]:
storage = Docker(
    registry_url="https://417497546600.dkr.ecr.us-east-2.amazonaws.com/get-tsx-moc-ecr",
    python_dependencies=["pandas", "sqlalchemy", "psycopg2", "boto3", "humps", "requests", "yfinance"],
    dockerfile=docker_flpth,
    image_name="etl-moc-img",
    image_tag="latest"
    )


In [13]:
storage.build()

[2020-04-22 16:26:24] INFO - prefect.Docker | Building the flow's Docker storage...


ConnectionError: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))

In [ ]:
etl_moc.etl_moc_flow.storage.build()

In [ ]:
# 1. Go to the UI client and create a project name

# 2. Register flow
## How to update flow? Without that error message?
etl_moc.etl_moc_flow.register(project_name="market-on-close")

In [ ]:
# tsxMocData = TsxMocData()

# moc_df = tsxMocData.scrape_moc_data()

## Load Raw Data

In [ ]:
# See a list of tsx moc data
moc_file_lst = get_moc_file_lst.run("tsx-moc")
moc_file_lst

In [ ]:
# Get all the files
data_flpth_lst = map(lambda flnm:  f"s3://{flnm}", moc_file_lst)
df_lst = [pd.read_csv(data_flpth, parse_dates=["moc_date"], na_filter=False) for data_flpth in list(data_flpth_lst)]
tsx_moc_df = pd.concat(df_lst , ignore_index=True)
tsx_moc_df.loc[tsx_moc_df["Symbol"]=="", "Symbol"] = "NA"

In [ ]:
tsx_moc_df[tsx_moc_df["Symbol"].isna()==True]

In [ ]:
tsx_moc_df["Symbol"].nunique()

## Create a key table
Map TSX symbols to Yahoo Symbols

In [ ]:
yhooMap =  MapTickerSymbols()
moc_key_df = yhooMap.run(tsx_moc_df)
moc_key_df["moc_date"] = moc_key_df["moc_date"].dt.date

In [ ]:
moc_key_df.info()

In [ ]:
#moc_key_df["moc_date"] = moc_key_df["moc_date"].dt.date

## Get daily features
(from yahoo)

#### Price

In [ ]:
eod_price_df = etl_moc.get_eod_price_data.run(moc_key_df)

#### Other Attributes
(float, short interest)

In [ ]:
eod_info_df = etl_moc.get_sym_info.run(moc_key_df)

## Get indtraday features
(from yahoo)

In [ ]:
intraday_df = etl_moc.get_1min_ohlc.run(moc_key_df)

## Create MOC table

In [ ]:
moc_df = etl_moc.build_moc_data.run(intraday_df, eod_price_df, eod_info_df)

In [ ]:
moc_df.head()

## Write to Db

In [ ]:
# 1. Eod
index_clmn_lst = ["moc_date", "yahoo_symbol"]

In [ ]:
num_rows_ins = etl_moc.df_to_db.run(intraday_df, tbl_name="intraday_prices", idx_clmn_lst=index_clmn_lst)

In [ ]:
# 2. Write to db
num_rows_ins = etl_moc.df_to_db.run(eod_price_df, tbl_name="eod_prices", idx_clmn_lst=index_clmn_lst)

In [ ]:
# 3. Write to db
num_rows_ins = etl_moc.df_to_db.run(eod_info_df, tbl_name="eod_sym_info", idx_clmn_lst=index_clmn_lst)

In [ ]:
# 4. Write to db
num_rows_ins = etl_moc.df_to_db.run(moc_df, tbl_name="daily_moc", idx_clmn_lst=index_clmn_lst)

In [ ]:
from prefect import task, Flow
from prefect.tasks.secrets import EnvVarSecret

@task
def print_value(x):
    print(x)

with Flow("Example") as flow:
    secret = EnvVarSecret("moc_pgdb_conn")
    print_value(secret)

In [ ]:
flow.run()

In [ ]:
import prefect
from prefect.client import Secret

In [ ]:
prefect.context.secrets["AWS_CREDENTIALS"] = 

In [ ]:
s = Secret("hello")

In [ ]:
s.exists()

In [ ]:
scrape_tsxmoc_fl.storage = Docker(dockerfile="/home/ilivni/MOC/Dockerfile")
scrape_tsxmoc_fl.register(project_name=project_nm)

In [ ]:
import boto3
session = boto3.session.Session()
client = session.client(
        service_name='secretsmanager'
    )
get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )

In [ ]:
secret = get_secret_value_response['SecretString']

In [ ]:
db_creds = get_db_creds()
type(db_creds)

In [ ]:
db_creds

In [ ]:
# Use this code snippet in your app.
# If you need more information about configurations or implementing the sample code, visit the AWS docs:
# https://aws.amazon.com/developers/getting-started/ruby/

# Use this code snippet in your app.
# If you need more information about configurations or implementing the sample code, visit the AWS docs:   
# https://aws.amazon.com/developers/getting-started/python/

import boto3
import base64
from botocore.exceptions import ClientError


def get_secret():

    secret_name = "moc-pg-db"
    region_name = "us-east-2"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    # In this sample we only handle the specific exceptions for the 'GetSecretValue' API.
    # See https://docs.aws.amazon.com/secretsmanager/latest/apireference/API_GetSecretValue.html
    # We rethrow the exception by default.

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        if e.response['Error']['Code'] == 'DecryptionFailureException':
            # Secrets Manager can't decrypt the protected secret text using the provided KMS key.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'InternalServiceErrorException':
            # An error occurred on the server side.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'InvalidParameterException':
            # You provided an invalid value for a parameter.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'InvalidRequestException':
            # You provided a parameter value that is not valid for the current state of the resource.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'ResourceNotFoundException':
            # We can't find the resource that you asked for.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
    else:
        # Decrypts secret using the associated KMS CMK.
        # Depending on whether the secret is a string or binary, one of these fields will be populated.
        if 'SecretString' in get_secret_value_response:
            secret = get_secret_value_response['SecretString']
        else:
            decoded_binary_secret = base64.b64decode(get_secret_value_response['SecretBinary'])
            
    # Your code goes here. 

In [ ]:
Mschedule = IntervalSchedule(
    start_date=datetime.utcnow() + timedelta(seconds=1),
    interval=timedelta(minutes=1),
)
schedule

In [ ]:
moc_df["Symbol"] =moc_df["Symbol"].fillna("NA")

In [ ]:
moc_df[moc_df["Symbol"].isnull()]

In [ ]:
moc_df.to_json(orient="records")